In [14]:
import os
import sys
import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
import joblib
import torch.nn.init as init
import seaborn as sns
import tslearn

from tslearn.metrics import cdist_dtw
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.special import softmax
from torch.utils.data import WeightedRandomSampler
from PIL import Image
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict

raw_vars = ["xeast", "ynorth", "zup", "vx", "vy", "vz", "head", "pitch", "roll"]
d1_vars = [i + "_d1" for i in raw_vars]

### Reading in Sample Manuevers Data

In [15]:
## will need to change directory to suit custom needs, this currently only works with tsv from sample maneuver data
home_dir = "/home/gridsan/dzhao/ManeuverID_shared/dan_zhao"
tsv_dir = home_dir + "/ManeuverID_archive/SampleManeuversData/SampleData/VanceAFB_tsv"
task_dir = home_dir + "/Task_2"

In [16]:
### 29 recognized maneuvers, 9-18 dimensions, ~2400 observed trajectrories (good+bad), at most 1.3e6 combinations/iterations

In [17]:
### read in data for categorized maneuvers, assuming each tsv describes the time-series of one maneuever each

if "recog_maneuvers.joblib" not in os.listdir(task_dir):
    recog_maneuvers = defaultdict()

    for m_name in os.listdir(tsv_dir):
        if m_name.endswith(".tsv"):
            m_key = m_name.replace(" ", "").split('_')[-1][:-8]
            recog_maneuvers[m_key] = -1
    i = 0
    for di in os.listdir(tsv_dir):
        if di.endswith(".tsv"):

            di_name = di.replace(" ", "").split('_')[-1][:-8]

            df_temp = pd.read_csv(tsv_dir + di, sep = '\t', index_col=0)
            df_temp.columns.values[0] = "time"
            df_temp.columns.values[1] = "xeast"
            df_temp.columns.values[2] = "ynorth"           
            df_temp.columns.values[3] = "zup"
            df_temp.columns.values[4] = "vx"
            df_temp.columns.values[5] = "vy"
            df_temp.columns.values[6] = "vz"           
            df_temp.columns.values[7] = "head"
            df_temp.columns.values[8] = "pitch"
            df_temp.columns.values[9] = "roll"


            for n, _ in enumerate(d1_vars):
                df_temp[d1_vars[n]] = df_temp[raw_vars[n]].diff()

            recog_maneuvers[di_name] = df_temp
    joblib.dump(recog_maneuvers, task_dir + "/recog_maneuvers.joblib", compress=3)

In [18]:
if "smpl_mnvrcorr.joblib" not in os.listdir(task_dir):
    recog_maneuvers = defaultdict()
    smpl_mnvrcorr = defaultdict()

    for k, v in recog_maneuvers.items():
        cor_raw = np.array(v[raw_vars].dropna().corr(method="kendall"))
        cor_d1 = np.array(v[d1_vars].dropna().corr(method="kendall"))

        smpl_mnvrcorr[k] = defaultdict()
        smpl_mnvrcorr[k]["raw"] = cor_raw
        smpl_mnvrcorr[k]["d1"] = cor_d1

    joblib.dump(smpl_mnvrcorr, task_dir + "/smpl_mnvrcorr.joblib", compress=3)

In [19]:
recog_maneuvers = joblib.load(task_dir + "/recog_maneuvers.joblib")
smpl_mnvrcorr = joblib.load(task_dir + "/smpl_mnvrcorr.joblib")

### Reading in Observed Trajectory Data

In [20]:
root_dir2 = home_dir+"/ManeuverID_archive/ObservedTrajectoryData/Sorted/path_tsvdata"
## using 1200... folder (can extend to 1300...)
good_dir = root_dir2 + "/12000000000_tsv_good" 
bad_dir = root_dir2 + "/12000000000_tsv_bad"

In [21]:
### mapping table for reference to tell what file (by file name) is in what class (good or bad)
good_fileidx = [f.replace(" ", "").split(".")[0] for f in os.listdir(good_dir)]
bad_fileidx = [f.replace(" ", "").split(".")[0] for f in os.listdir(bad_dir)]

In [22]:
if "obs_trajcorr.joblib" not in os.listdir(task_dir):

    obs_trajcorr = defaultdict()

    result_good = pd.DataFrame()
    result_bad = pd.DataFrame()

    covs_good = []
    covs_bad = []

    t1 = time.time()
    i = 0
    for di in os.listdir(good_dir):
        if di.endswith(".tsv"):

            df_temp = pd.read_csv(good_dir + di, sep = '\t', index_col=0)
            df_temp.columns.values[0] = "time"
            df_temp.columns.values[1] = "xeast"
            df_temp.columns.values[2] = "ynorth"           
            df_temp.columns.values[3] = "zup"
            df_temp.columns.values[4] = "vx"
            df_temp.columns.values[5] = "vy"
            df_temp.columns.values[6] = "vz"           
            df_temp.columns.values[7] = "head"
            df_temp.columns.values[8] = "pitch"
            df_temp.columns.values[9] = "roll"

            for n, _ in enumerate(d1_vars):
                df_temp[d1_vars[n]] = df_temp[raw_vars[n]].diff()

            cor_raw = np.array(df_temp[raw_vars].dropna().corr(method="kendall"))
            cor_d1 = np.array(df_temp[d1_vars].dropna().corr(method="kendall"))

    #         covs_good.append(cov)

            df_temp["obsTrajID"] = i
            df_temp["TrajClass"] = "good"
            file_idx = di.replace(" ", "").split(".")[0]
            df_temp["fileIdx"] = file_idx

            obs_trajcorr[file_idx] = defaultdict()
            obs_trajcorr[file_idx]["raw"] = cor_raw                         
            obs_trajcorr[file_idx]["d1"] = cor_d1                                                               

            result_good = pd.concat([result_good, df_temp])

            i+=1

    i = 0
    for di in os.listdir(bad_dir):
        if di.endswith(".tsv"):

            df_temp = pd.read_csv(bad_dir + di, sep = '\t', index_col=0)
            df_temp.columns.values[0] = "time"
            df_temp.columns.values[1] = "xeast"
            df_temp.columns.values[2] = "ynorth"           
            df_temp.columns.values[3] = "zup"
            df_temp.columns.values[4] = "vx"
            df_temp.columns.values[5] = "vy"
            df_temp.columns.values[6] = "vz"           
            df_temp.columns.values[7] = "head"
            df_temp.columns.values[8] = "pitch"
            df_temp.columns.values[9] = "roll"

            for n, _ in enumerate(d1_vars):
                df_temp[d1_vars[n]] = df_temp[raw_vars[n]].diff()

            cor_raw = np.array(df_temp[raw_vars].dropna().corr(method="kendall"))
            cor_d1 = np.array(df_temp[d1_vars].dropna().corr(method="kendall"))

    #         covs_bad.append(cov)   

            df_temp["obsTrajID"] = i ## id within good/bad class
            df_temp["TrajClass"] = "bad" ## good or bad
            file_idx = di.replace(" ", "").split(".")[0] ## corresponding file id                           
            df_temp["fileIdx"] = file_idx

            obs_trajcorr[file_idx] = defaultdict()
            obs_trajcorr[file_idx]["raw"] = cor_raw                         
            obs_trajcorr[file_idx]["d1"] = cor_d1                                                               

            result_bad = pd.concat([result_bad, df_temp])

            i+=1
        
    t2 = time.time()
    print(t2-t1) ## takes about 830 seconds to run through cell (13 min, likely due to kendall corr mats)
    
    
    result_good.reset_index(inplace = True, drop=True)
    result_bad.reset_index(inplace = True, drop=True)
    
    ### combine good and bad observed trajectories into one dataset
    allobs_trajectories = pd.concat([result_good, result_bad])
    allobs_trajectories.reset_index(inplace=True)
    
    allobs_trajectories.to_csv(task_dir + "/tabdf_all.csv", sep = '\t', index=False)
    result_good.to_csv(task_dir + "/good_tabdf_all.csv", sep = '\t', index=False)
    result_bad.to_csv(task_dir + "/bad_tabdf_all.tsv", sep = '\t', index=False)
    
    joblib.dump(obs_trajcorr, "obs_trajcorr.joblib", compress=3)

    
result_good = pd.read_csv(task_dir+"/good_tabdf_all.csv")
result_bad = pd.read_csv(task_dir+"/bad_tabdf_all.csv")
allobs_trajectories = pd.read_csv(task_dir+"/tabdf_all.tsv", sep = "\t") 
obs_trajcorr = joblib.load(task_dir+"/obs_trajcorr.joblib")

### time-series simlarity & profiling

In [23]:
all_ids = list(allobs_trajectories["fileIdx"].astype(str).unique())

### mappings to help with looking up results array
smplmnvrID_map = dict(zip(list(recog_maneuvers.keys()), np.arange(len(recog_maneuvers))))
obstrajID_map = dict(zip(list(all_ids), np.arange(len(all_ids))))

# obstrajID_map["12000004003"]

In [24]:
# raw_vars = ["xeast", "ynorth", "zup", "vx", "vy", "vz", "head", "pitch", "roll"]
raw_vars2 = ["vx", "vy", "vz", "head", "pitch", "roll"]
d1_vars2 = [i + "_d1" for i in raw_vars2]

In [25]:
def corr_sim(A,B):
#     d = 1 - np.trace(A @ B) / (np.linalg.norm(A, ord="fro") * np.linalg.norm(B, ord="fro"))
    d = np.trace(A @ B) / (np.sum(A ** 2) ** 0.5 * np.sum(B ** 2) ** 0.5)
    return d

def KL(A,B):
    KL_ab = 0.5 * (np.trace(np.linalg.pinv(A) @ B) - np.log((np.linalg.det(B)/(np.linalg.det(A) + 1e-5)) + 1e-5))
    KL_ba = 0.5 * (np.trace(np.linalg.pinv(B) @ A) - np.log((np.linalg.det(A)/(np.linalg.det(B) + 1e-5)) + 1e-5))
    KL_sym = 0.5 * KL_ab + 0.5 * KL_ba
    return KL_sym

In [67]:
len(obs_trajcorr.keys())

2745

### correlation distances

In [81]:
allobs_trajectories["fileIdx"] = allobs_trajectories["fileIdx"].astype(str)

ct = 0
# univar_simraw = np.zeros((len(smplmnvrID_map), len(obstrajID_map))).T ## number of fileIds (rows) by number of maneuvers (cols)
# univar_simd1 = np.zeros((len(smplmnvrID_map), len(obstrajID_map))).T ## number of fileIds (rows) by number of maneuvers (cols)
# multivar_simraw = np.zeros((len(smplmnvrID_map), len(obstrajID_map))).T ## number of fileIds (rows) by number of maneuvers (cols)
# multivar_simd1 = np.zeros((len(smplmnvrID_map), len(obstrajID_map))).T ## number of fileIds (rows) by number of maneuvers (cols)

multivar_simraw = defaultdict(str)
multivar_simd1 = defaultdict(str)

for f in list(obs_trajcorr.keys()):
    multivar_simraw[f] = {}
    multivar_simd1[f] = {}
    for i in list(recog_maneuvers.keys()):
        multivar_simraw[f][i] = -999
        multivar_simraw[f][i] = -999  

assert len(raw_vars2) == len(d1_vars2)

t1 = time.time()
for n, k in enumerate(recog_maneuvers.keys()):
#     manvr_dfraw = np.array(recog_maneuvers[k][raw_vars2].dropna()) ## fetch maneuver profile and iterate overall maneuver ids
#     manvr_dfd1 = np.array(recog_maneuvers[k][d1_vars2].dropna())    

#     try:
#         adjst = (np.max(manvr_dfraw, axis=0) + 1e-5)
#     except ValueError:  #raised if error
#         print(k)
#         manvr_dfraw = manvr_dfraw
#     else:
#         manvr_dfraw = manvr_dfraw/adjst ## 1e-5 to prevent overflow/nan issues
        
#     try:
#         adjst = (np.max(manvr_dfd1, axis=0) + 1e-5)
#     except ValueError:  #raised if is empty.
#         print(k)
#         manvr_dfd1 = manvr_dfd1
#     else:
#         manvr_dfd1 = manvr_dfd1/adjst ## 1e-5 to prevent overflow/nan issues
    
    mvr_corr_raw = smpl_mnvrcorr[k]["raw"]
    mvr_corr_d1 = smpl_mnvrcorr[k]["d1"]
        
    for num, obsID in enumerate(list(obs_trajcorr.keys())):

#         if num % 1300 == 0: print(str(n) + ':' + str(num))
        ## quantify joint similarities
        
        obs_corr_raw = obs_trajcorr[obsID]["raw"] 
        obs_corr_d1 = obs_trajcorr[obsID]["d1"]
        
        mvr_corr_raw[np.isnan(mvr_corr_raw)] = 0
        mvr_corr_d1[np.isnan(mvr_corr_d1)] = 0
        obs_corr_raw[np.isnan(obs_corr_raw)] = 0
        obs_corr_d1[np.isnan(obs_corr_d1)] = 0
                
#         multivar_simraw[obstrajID_map[obsID], smplmnvrID_map[k]] = KL(mvr_corr_raw, obs_corr_raw)
#         multivar_simd1[obstrajID_map[obsID], smplmnvrID_map[k]] = KL(mvr_corr_d1, obs_corr_d1)
        multivar_simraw[obsID][k] = KL(mvr_corr_raw, obs_corr_raw)
        multivar_simd1[obsID][k] = KL(mvr_corr_d1, obs_corr_d1)         
        
##############

#         ## quantify marginal similarity (tslearn)
    
#         univar_simraw[obstrajID_map[obsID], smplmnvrID_map[k]] = cdist_dtw(np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][raw_vars2]), 
#                              manvr_dfraw,
#                              global_constraint="itakura", n_jobs=-1)

#         univar_simd1[obstrajID_map[obsID], smplmnvrID_map[k]] = cdist_dtw(np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][d1_vars2]), 
#                              manvr_dfd1,
#                              global_constraint="itakura", n_jobs=-1)

#         ## quantify marginal similarity (fast dtw)
    
#         obstraj_temp = np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][raw_vars2].dropna())            
# #         adjst = (np.max(obstraj_temp, axis=0) + 1e-5)            
#         try:
#             np.max(obstraj_temp, axis=0)
#         except ValueError:  #raised if error
#             print(obsID)
#             obstraj_temp = obstraj_temp
#         else:
#             obstraj_temp = obstraj_temp/(np.max(obstraj_temp, axis=0) + 1e-5) ## 1e-5 to prevent overflow/nan issues    

#         a, _ = fastdtw(obstraj_temp, manvr_dfraw, dist=euclidean)
#         univar_simraw[obstrajID_map[obsID],smplmnvrID_map[k]] = a


#         obstraj_temp = np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][d1_vars2].dropna())
# #         adjst = (np.max(obstraj_temp, axis=0) + 1e-5)

#         try:
#             np.max(obstraj_temp, axis=0)
#         except ValueError:  #raised if error
#             print(obsID)
#             obstraj_temp = obstraj_temp
#         else:
#             obstraj_temp = obstraj_temp/(np.max(obstraj_temp, axis=0) + 1e-5) ## 1e-5 to prevent overflow/nan issues    


#         b , _ = fastdtw(obstraj_temp, manvr_dfd1, dist=euclidean)
#         univar_simd1[obstrajID_map[obsID],smplmnvrID_map[k]] = b
        
#     joblib.dump([univar_simraw, univar_simd1], 
#                 "/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/univarsim/univar_sims_"+ str(n) + ".joblib", compress=3)
        
joblib.dump(multivar_simraw, task_dir+"/multivar_simraw.joblib", compress=3)
joblib.dump(multivar_simd1, task_dir+"/multivar_simd1.joblib", compress=3)

# print(time.time() - t1) ## about 325.52 seconds (5 min) with fast DTW algo
print("done,", str(time.time() - t1)) 

done, 25.654815435409546


### time-series distances

In [ ]:
# raw_vars = ["xeast", "ynorth", "zup", "vx", "vy", "vz", "head", "pitch", "roll"]

allobs_trajectories["fileIdx"] = allobs_trajectories["fileIdx"].astype(str)

univar_simraw = defaultdict(str)
univar_simd1 = defaultdict(str)

for f in list(obs_trajcorr.keys()):
    univar_simraw[f] = {}
    univar_simd1[f] = {}
    for i in list(recog_maneuvers.keys()):
        univar_simraw[f][i] = -999
        univar_simd1[f][i] = -999  

# track = []

assert len(raw_vars2) == len(d1_vars2)

t1 = time.time()
for num, obsID in enumerate(list(obs_trajcorr.keys())):
# for num, obsID in enumerate(list(obs_trajcorr.keys())[58:]):
    if num % 200 == 0: print(obsID)
    
#     univar_simraw = np.zeros((len(smplmnvrID_map), 1)).T ## number of fileIds (rows) by number of maneuvers (cols)
#     univar_simd1 = np.zeros((len(smplmnvrID_map), 1)).T ## number of fileIds (rows) by number of maneuvers (cols)
    
    for n, k in enumerate(recog_maneuvers.keys()):
        manvr_dfraw = np.array(recog_maneuvers[k][raw_vars2].dropna()) ## fetch maneuver profile and iterate overall maneuver ids
        manvr_dfd1 = np.array(recog_maneuvers[k][d1_vars2].dropna())    

        try:
            adjst = (np.max(manvr_dfraw, axis=0) + 1e-5)
        except ValueError:  #raised if error
            print(k)
            manvr_dfraw = manvr_dfraw
        else:
            manvr_dfraw = manvr_dfraw/adjst ## 1e-5 to prevent overflow/nan issues

        try:
            adjst = (np.max(manvr_dfd1, axis=0) + 1e-5)
        except ValueError:  #raised if is empty.
            print(k)
            manvr_dfd1 = manvr_dfd1
        else:
            manvr_dfd1 = manvr_dfd1/adjst ## 1e-5 to prevent overflow/nan issues    


        obstraj_temp = np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][raw_vars2].dropna())            
    #         adjst = (np.max(obstraj_temp, axis=0) + 1e-5)            
        try:
            np.max(obstraj_temp, axis=0)
        except ValueError:  #raised if error
            print(obsID)
            obstraj_temp = obstraj_temp
        else:
            obstraj_temp = obstraj_temp/(np.max(obstraj_temp, axis=0) + 1e-5) ## 1e-5 to prevent overflow/nan issues    

        a, _ = fastdtw(obstraj_temp, manvr_dfraw, dist=euclidean)
#         univar_simraw[0, smplmnvrID_map[k]] = a
        univar_simraw[obsID][k] = a


        obstraj_temp = np.array(allobs_trajectories[allobs_trajectories["fileIdx"] == obsID][d1_vars2].dropna())
    #         adjst = (np.max(obstraj_temp, axis=0) + 1e-5)
        try:
            np.max(obstraj_temp, axis=0)
        except ValueError:  #raised if error
            print(obsID)
            obstraj_temp = obstraj_temp
        else:
            obstraj_temp = obstraj_temp/(np.max(obstraj_temp, axis=0) + 1e-5) ## 1e-5 to prevent overflow/nan issues    

        b , _ = fastdtw(obstraj_temp, manvr_dfd1, dist=euclidean)
#         univar_simd1[0, smplmnvrID_map[k]] = b
        univar_simd1[obsID][k] = b
    
#     track.append(obsID)
#     joblib.dump(univar_simraw, 
#                 "/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/univarsim/univar_simsraw_" + str(obsID) + ".joblib", 
#                 compress=3)
#     joblib.dump(univar_simd1, 
#                 "/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/univarsim/univar_simsd1_" + str(obsID) + ".joblib", 
#                 compress=3)
#     joblib.dump(track, 
#             "/home/gridsan/DA30449/ManeuverID/ObservedTrajectoryData/univarsim/tracker.joblib", 
#             compress=3)

    joblib.dump(univar_simraw, task_dir + "/univar_simraw.joblib", compress=3)
    joblib.dump(univar_simd1, task_dir + "/univar_simd1.joblib", compress=3)    
                                          
print(time.time() - t1) ## about 325.52 seconds (5 min) with fast DTW algo

### similarity scoring

In [26]:
univar_simd1 = joblib.load("univar_simd1.joblib")
univar_simraw = joblib.load("univar_simraw.joblib")
multivar_simd1 = joblib.load("multivar_simd1.joblib")
multivar_simraw = joblib.load("multivar_simraw.joblib")

In [76]:
cmn_keys = list(set(list(univar_simd1.keys()) + list(univar_simraw.keys()) + list(multivar_simd1.keys()) + list(multivar_simraw.keys())))
mvr_names = list(univar_simd1['12002670003'].keys())
scores = defaultdict()

for i in cmn_keys:
    
    if -999 in list(univar_simd1[i].values()) or -999 in list(univar_simraw[i].values()):
        pass
    else:
        scores[i] = defaultdict()
        A = softmax(np.array(list(univar_simd1[i].values())))
        B = softmax(np.array(list(univar_simraw[i].values())))
        C = softmax(np.array(list(multivar_simd1[i].values())))
        D = softmax(np.array(list(multivar_simraw[i].values())))

        scores[i]["scoresA"] = A
        scores[i]["scoresB"] = B
        scores[i]["scoresC"] = C
        scores[i]["scoresD"] = D        

In [77]:
scores

defaultdict(None,
            {'12002619003': defaultdict(None,
                         {'scoresA': array([3.32041374e-138, 8.78615730e-135, 8.74930630e-103, 1.00000000e+000,
                                 9.44980719e-182, 4.20124631e-219, 4.64147129e-241, 9.40069719e-192,
                                 6.44712207e-246, 5.49580548e-271, 6.52052732e-187, 5.84065677e-132,
                                 5.87603292e-084, 1.92589325e-113, 8.06292700e-113, 6.52654771e-244,
                                 1.85599226e-243, 2.05898105e-111, 3.55405857e-163, 4.37045111e-208,
                                 4.97279236e-206, 4.87051965e-183, 1.10716773e-134, 4.55782037e-194,
                                 5.69558822e-158, 1.07708864e-192, 7.42964797e-216, 5.52758681e-058,
                                 6.64370177e-155]),
                          'scoresB': array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                                 0., 0., 0., 0., 0., 0

In [67]:
c=0
err = []
for k,v in univar_simd1.items():
    if -999 in list(v.values()):
        c+=1
        err.append(k)

In [74]:
foo = []
for i in err:
    if i not in os.listdir("/home/gridsan/dzhao/ManeuverID_shared/dan_zhao/ManeuverID_archive/ObservedTrajectoryData/Sorted/path_tsvdata/12000000000_tsv_bad"):
        foo.append(i)

In [56]:
scores['12002670003']["scoresC"]

array([0.02237977, 0.01803044, 0.03328986, 0.09392493, 0.01938941,
       0.02260443, 0.02519771, 0.01989464, 0.01446206, 0.02563885,
       0.03298281, 0.01159734, 0.01896849, 0.0602315 , 0.11449687,
       0.01535016, 0.05990338, 0.01154179, 0.01321367, 0.03654596,
       0.01530546, 0.04551086, 0.01881962, 0.0139696 , 0.01289314,
       0.12535714, 0.02918552, 0.04133047, 0.02798412])

In [32]:
row_list = []
for key_id in cmn_keys:
    univar_simd1 
    
df = pd.DataFrame(rows_list)

In [35]:
univar_simd1['12002670003'].keys()

dict_keys(['AileronRoll', 'ILS', 'UnusualAttitudeNoseHigh', 'UnusualAttitudeNoseLow', 'StraightIn', 'NoseLowRecovery', 'OverheadPattern', 'ELP-PEL', 'SplitS', 'ClosedPullup', 'ELP-FL', 'SlowFlight', 'PowerOnStallNoseHigh', 'VerticalSbravo', 'Lazy8', '60SteepTurn', 'BarrelRoll', 'LandingAttitudeTPStall', 'UndershootingTPStall', 'Immelman', 'IntentionalSpin', 'Cuban8', 'Localizer', 'OvershootingTPStall', 'PowerOnStallNLTurning', 'Loop', 'NoseHighRecovery', '45SteepTurn', 'VerticalSalpha'])